# SSL pretext-task: masking reconstruction

## Загрузка и подготовка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ==== Загрузка маленького датасета ==== #

import numpy as np
import pandas as pd

# Путь к подготовленной маленькой выборке на диске
data_path = '/content/drive/MyDrive/MIPT/Diploma_P300/Data/ssl_data_50k.npz'

data = np.load(data_path)
X_ssl_ready = data["X"].astype(np.float32)   # (50000, 14, 208)
ssl_mean    = data["mean"].astype(np.float32)
ssl_std     = data["std"].astype(np.float32)

print(X_ssl_ready.shape, X_ssl_ready.dtype)


(50000, 14, 208) float32


In [3]:
# ==== Делим выборку на train / val ==== #

N = X_ssl_ready.shape[0]
rng = np.random.default_rng(42)
idx = np.arange(N)
rng.shuffle(idx)

val_ratio = 0.1
n_val = int(N * val_ratio)

idx_val = idx[:n_val]
idx_train = idx[n_val:]

X_train = X_ssl_ready[idx_train]
X_val   = X_ssl_ready[idx_val]

print("Train:", X_train.shape, "Val:", X_val.shape)


Train: (45000, 14, 208) Val: (5000, 14, 208)


In [4]:
# Настраиваем PyTorch и device
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [5]:
# ==== Dataset ==== #
import torch
from torch.utils.data import Dataset

class SSLDataset(Dataset):
    """
    Простой датасет для SSL реконструкции:
    хранит массив эпох (N, C, L) и
    возвращает ТОЛЬКО один тензор x.
    """
    def __init__(self, X):
        """
        X — numpy array или torch.Tensor формы (N, C, L)
        """
        if isinstance(X, torch.Tensor):
            self.X = X.float()
        else:
            # numpy → torch
            self.X = torch.tensor(X, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Возвращаем только один тензор (C, L)
        return self.X[idx]



In [6]:
train_dataset = SSLDataset(X_train)
val_dataset   = SSLDataset(X_val)

print("Размер тренировочного датасета:", len(train_dataset))
print("Размер валидационного датасета:", len(val_dataset))



Размер тренировочного датасета: 45000
Размер валидационного датасета: 5000


In [7]:
# ==== DataLoader ==== #

if device.type == "cuda":
    BATCH_SIZE = 64        # наверно можно больше
    NUM_WORKERS = 0
    PIN_MEMORY = True
else:
    BATCH_SIZE = 16        # на CPU поменьше
    NUM_WORKERS = 0        # чтобы не было багов
    PIN_MEMORY = False

print("batch_size:", BATCH_SIZE,
      "| num_workers:", NUM_WORKERS,
      "| pin_memory:", PIN_MEMORY)


batch_size: 64 | num_workers: 0 | pin_memory: True


In [8]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)

## Реализация time-masking

- маскируем по времени (ось L) все каналы одновременно,
- примерно 50% длины эпохи обнуляем,
- каналы НЕ трогаем (0% channel masking).

In [9]:
import torch

def time_mask(x: torch.Tensor,
              mask_ratio: float = 0.5,
              min_width: int = 5,
              max_width: int | None = None) -> torch.Tensor:
    """
    Time masking.

    x:          (B, C, L) — батч ЭЭГ-эпох после нормализации.
    mask_ratio: доля временных отсчётов, которую хотим замаскировать (0.5 → 50%).
    min_width:  минимальная длина одного маскируемого отрезка (в шагах по времени).
    max_width:  максимальная длина одного отрезка; если None → L // 4.

    Возвращает:
        x_tilde той же формы (B, C, L) с нулями на замаскированных участках.
    """
    assert x.dim() == 3, "Ожидается тензор формы (B, C, L)"
    B, C, L = x.shape
    x_cor = x.clone()

    if max_width is None:
        max_width = max(min_width, L // 4)

    # Для каждого примера в батче генерируем свою маску
    for b in range(B):
        total_to_mask = int(mask_ratio * L)
        masked = 0

        while masked < total_to_mask:
            # случайная ширина отрезка
            width = torch.randint(min_width, max_width + 1, (1,)).item()
            width = min(width, total_to_mask - masked)  # чтобы не перебрать сильно

            # случайная позиция отрезка
            if width >= L:
                start = 0
            else:
                start = torch.randint(0, L - width + 1, (1,)).item()
            end = start + width

            # Маскируем по всем каналам одновременно
            x_cor[b, :, start:end] = 0.0

            masked += width

    return x_cor


## SSL-loss «time–frequency consistency»

In [10]:
# Временная часть L1
import torch.nn.functional as F

def ssl_time_loss(x, x_hat):
    """
    L_time = ||x - x_hat||_1
    x, x_hat: (B, C, L)
    """
    return F.l1_loss(x_hat, x)


In [11]:
# Частотная часть L1

import torch

def ssl_freq_loss(x, x_hat):
    """
    L_freq = ||FFT(x) - FFT(x_hat)||_1 по амплитуде спектра.

    x, x_hat: (B, C, L), вещественные тензоры.
    FFT считаем по временной оси (последний индекс).
    """
    # Прямое БПФ по времени
    X_f = torch.fft.rfft(x, dim=-1)       # (B, C, L_freq)
    Xhat_f = torch.fft.rfft(x_hat, dim=-1)

    # Амплитуды (модули комплексных чисел)
    X_mag = torch.abs(X_f)
    Xhat_mag = torch.abs(Xhat_f)

    # L1 по амплитудам
    return F.l1_loss(Xhat_mag, X_mag)


In [12]:
# Общий Loss
def ssl_time_freq_loss(x, x_hat, lambda_fft: float = 0.1):
    """
    Полный SSL-loss:
        L_SSL = L_time + lambda_fft * L_freq

    x, x_hat: (B, C, L)
    lambda_fft: вес частотной компоненты.
    """
    L_time = ssl_time_loss(x, x_hat)
    L_freq = ssl_freq_loss(x, x_hat)

    L_ssl = L_time + lambda_fft * L_freq
    return L_ssl, L_time, L_freq


## Модель SSL: 1D U-Net

In [14]:
import torch.nn as nn

class DoubleConv1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)


class Down1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.MaxPool1d(kernel_size=2, stride=2),
            DoubleConv1D(in_channels, out_channels)
        )

    def forward(self, x):
        return self.block(x)


class Up1D(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.bilinear = bilinear

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="linear", align_corners=True)
            mid_channels = in_channels // 2
            self.conv = DoubleConv1D(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose1d(in_channels, out_channels, kernel_size=2, stride=2)
            self.conv = DoubleConv1D(in_channels, out_channels)

    def forward(self, x1, x2):
        # x1: низ, x2: skip
        x1 = self.up(x1)

        # подгоняем длину, если не совпадает
        diff = x2.size(-1) - x1.size(-1)
        if diff > 0:
            x1 = nn.functional.pad(x1, (diff // 2, diff - diff // 2))
        elif diff < 0:
            x2 = nn.functional.pad(x2, (-diff // 2, -diff - (-diff // 2)))

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet1D_Light(nn.Module):
    """
    U-Net 1D с 4 уровнями даунсемплинга, вдохновлён Hong et al., 2025.
    Каналы: 32 → 64 → 128 → 256, bottleneck 512.
    """
    def __init__(self, n_channels, n_classes, base_ch=32, bilinear=True):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        ch1 = base_ch
        ch2 = base_ch * 2
        ch3 = base_ch * 4
        ch4 = base_ch * 8
        bottleneck_ch = base_ch * 16  # 512 при base_ch=32

        # encoder
        self.inc = DoubleConv1D(n_channels, ch1)
        self.down1 = Down1D(ch1, ch2)
        self.down2 = Down1D(ch2, ch3)
        self.down3 = Down1D(ch3, ch4)
        self.down4 = Down1D(ch4, bottleneck_ch)

        # decoder
        self.up1 = Up1D(bottleneck_ch + ch4, ch4, bilinear)
        self.up2 = Up1D(ch4 + ch3, ch3, bilinear)
        self.up3 = Up1D(ch3 + ch2, ch2, bilinear)
        self.up4 = Up1D(ch2 + ch1, ch1, bilinear)
        self.outc = OutConv1D(ch1, n_classes)

    def encode(self, x):
        x1 = self.inc(x)     # (N, ch1, L)
        x2 = self.down1(x1)  # (N, ch2, L/2)
        x3 = self.down2(x2)  # (N, ch3, L/4)
        x4 = self.down3(x3)  # (N, ch4, L/8)
        x5 = self.down4(x4)  # (N, bottleneck_ch, L/16)
        return x1, x2, x3, x4, x5

    def decode(self, x1, x2, x3, x4, x5):
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def forward(self, x):
        x1, x2, x3, x4, x5 = self.encode(x)
        logits = self.decode(x1, x2, x3, x4, x5)
        return logits, x5  # x5 — bottleneck


In [19]:
import os

# Гиперпараметры
CFG = {
    "epochs": 200,             # можно начать с 10–20 для теста, в статье 200
    "batch_size": 64,
    "lr_init": 2.5e-4,
    "max_lr": 5e-4,
    "weight_decay": 1e-2,
    "lambda_fft": 0.1,         # вес частотной части loss
    "warmup_pct": 0.1,         # 10% шагов на warmup
    "div_factor": 2.0,         # max_lr / div_factor = initial_lr (≈2.5e-4)
    "final_div_factor": 100.0, # final_lr ≈ initial_lr / 100 ≈ 2.5e-6
    "patience": 15,            # early stopping по эпохам
    "save_dir": "/content/drive/MyDrive/MIPT/Diploma_P300/SSL",
    "model_name": "unet_ssl_maskrecon_50k_200epoch",
}

os.makedirs(CFG["save_dir"], exist_ok=True)
best_ckpt_path = os.path.join(CFG["save_dir"], CFG["model_name"] + "_best.pth")

In [20]:
# Инициализация
unet = UNet1D_Light(
    n_channels=14,
    n_classes=14, # reconstruction по всем каналам
    base_ch=32,
).to(device)

print(sum(p.numel() for p in unet.parameters()) / 1e6, "M parameters")


2.62491 M parameters


In [21]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

# ==== Оптимизатор AdamW ====

optimizer = AdamW(
    unet.parameters(),
    lr=CFG["lr_init"],
    weight_decay=CFG["weight_decay"]
)

# ==== OneCycleLR (по мотивам статьи) ====

steps_per_epoch = len(train_loader)
scheduler = OneCycleLR(
    optimizer,
    max_lr=CFG["max_lr"],
    epochs=CFG["epochs"],
    steps_per_epoch=steps_per_epoch,
    pct_start=CFG["warmup_pct"],
    div_factor=CFG["div_factor"],
    final_div_factor=CFG["final_div_factor"],
    anneal_strategy="cos",
)

## train+val с time-masking и time–freq loss

In [22]:
def train_ssl_epoch(model, train_loader, optimizer, scheduler, device, lambda_fft: float):
    model.train()
    running_ssl, running_time, running_freq = 0.0, 0.0, 0.0
    n_batches = len(train_loader)

    pbar = tqdm(train_loader, desc="Train", leave=False)
    for x in pbar:
        x = x.to(device)                     # (B, C, L) — чистый сигнал

        # 1) Маскируем по времени (50% time masking)
        x_tilde = time_mask(x, mask_ratio=0.5)

        # 2) Прогоняем замаскированный сигнал через U-Net
        x_hat, _ = model(x_tilde)           # x_hat: (B, C, L)

        # 3) Считаем time–freq SSL-loss
        L_ssl, L_time, L_freq = ssl_time_freq_loss(
            x=x,
            x_hat=x_hat,
            lambda_fft=lambda_fft
        )

        optimizer.zero_grad()
        L_ssl.backward()
        optimizer.step()
        scheduler.step()

        running_ssl += L_ssl.item()
        running_time += L_time.item()
        running_freq += L_freq.item()

        pbar.set_postfix({
            "L_ssl": f"{L_ssl.item():.4f}",
            "L_t": f"{L_time.item():.4f}",
            "L_f": f"{L_freq.item():.44f}",
        })

    avg_ssl = running_ssl / n_batches
    avg_time = running_time / n_batches
    avg_freq = running_freq / n_batches
    return avg_ssl, avg_time, avg_freq


@torch.no_grad()
def val_ssl_epoch(model, val_loader, device, lambda_fft: float):
    model.eval()
    running_ssl, running_time, running_freq = 0.0, 0.0, 0.0
    n_batches = len(val_loader)

    for x in val_loader:
        x = x.to(device)

        # Валидация: тоже маскируем, чтобы метрика соответствовала задаче
        x_tilde = time_mask(x, mask_ratio=0.5)

        x_hat, _ = model(x_tilde)

        L_ssl, L_time, L_freq = ssl_time_freq_loss(
            x=x,
            x_hat=x_hat,
            lambda_fft=lambda_fft
        )

        running_ssl += L_ssl.item()
        running_time += L_time.item()
        running_freq += L_freq.item()

    avg_ssl = running_ssl / n_batches
    avg_time = running_time / n_batches
    avg_freq = running_freq / n_batches
    return avg_ssl, avg_time, avg_freq


## Цикл обучения


In [23]:
from tqdm.auto import tqdm

best_val_ssl = float("inf")
epochs_no_improve = 0

history = {
    "train_ssl": [],
    "train_time": [],
    "train_freq": [],
    "val_ssl": [],
    "val_time": [],
    "val_freq": [],
}

for epoch in range(1, CFG["epochs"] + 1):
    print(f"\nEpoch [{epoch}/{CFG['epochs']}]")

    # ---- Train ----
    train_ssl, train_time, train_freq = train_ssl_epoch(
        model=unet,
        train_loader=train_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        lambda_fft=CFG["lambda_fft"],
    )

    # ---- Validation ----
    val_ssl, val_time, val_freq = val_ssl_epoch(
        model=unet,
        val_loader=val_loader,
        device=device,
        lambda_fft=CFG["lambda_fft"],
    )

    # Логируем
    history["train_ssl"].append(train_ssl)
    history["train_time"].append(train_time)
    history["train_freq"].append(train_freq)
    history["val_ssl"].append(val_ssl)
    history["val_time"].append(val_time)
    history["val_freq"].append(val_freq)

    print(
        f"  Train: L_ssl={train_ssl:.4f} | L_time={train_time:.4f} | L_freq={train_freq:.4f}\n"
        f"  Val:   L_ssl={val_ssl:.4f} | L_time={val_time:.4f} | L_freq={val_freq:.4f}"
    )

    # ---- Early stopping по val_ssl ----
    if val_ssl < best_val_ssl - 1e-5:   # небольшой delta, чтобы игнорировать шум
        best_val_ssl = val_ssl
        epochs_no_improve = 0

        torch.save(unet.state_dict(), best_ckpt_path)
        print(f"  >> New best model saved! val_ssl={best_val_ssl:.6f}")
    else:
        epochs_no_improve += 1
        print(f"  No improvement for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= CFG["patience"]:
        print(f"\nEarly stopping triggered after {epoch} epochs.")
        break

print("\nTraining finished.")
print("Best val_ssl:", best_val_ssl)
print("Best checkpoint path:", best_ckpt_path)



Epoch [1/200]


Train:   0%|          | 0/703 [00:00<?, ?it/s]

  Train: L_ssl=0.6544 | L_time=0.4066 | L_freq=2.4779
  Val:   L_ssl=0.5443 | L_time=0.3347 | L_freq=2.0962
  >> New best model saved! val_ssl=0.544341

Epoch [2/200]


Train:   0%|          | 0/703 [00:00<?, ?it/s]

  Train: L_ssl=0.5314 | L_time=0.3254 | L_freq=2.0606
  Val:   L_ssl=0.5156 | L_time=0.3135 | L_freq=2.0207
  >> New best model saved! val_ssl=0.515606

Epoch [3/200]


...

  Train: L_ssl=0.3337 | L_time=0.1941 | L_freq=1.3952
  Val:   L_ssl=0.3292 | L_time=0.1898 | L_freq=1.3938
  No improvement for 8 epoch(s).

Epoch [199/200]


Train:   0%|          | 0/703 [00:00<?, ?it/s]

  Train: L_ssl=0.3338 | L_time=0.1942 | L_freq=1.3959
  Val:   L_ssl=0.3317 | L_time=0.1907 | L_freq=1.4101
  No improvement for 9 epoch(s).

Epoch [200/200]


Train:   0%|          | 0/703 [00:00<?, ?it/s]

  Train: L_ssl=0.3342 | L_time=0.1945 | L_freq=1.3967
  Val:   L_ssl=0.3295 | L_time=0.1897 | L_freq=1.3982
  No improvement for 10 epoch(s).

Training finished.
Best val_ssl: 0.3281184683872175
Best checkpoint path: /content/drive/MyDrive/MIPT/Diploma_P300/SSL/unet_ssl_maskrecon_50k_200epoch_best.pth
